In [1]:
import numpy as np
import tensorflow as tf
from sklearn.utils import shuffle
import re
import time
import collections
import os
import itertools
from sklearn.cross_validation import train_test_split

/usr/local/lib/python3.6/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
def build_dataset(words, n_words, atleast=1):
    count = [['GO', 0], ['PAD', 1], ['EOS', 2], ['UNK', 3]]
    counter = collections.Counter(words).most_common(n_words)
    counter = [i for i in counter if i[1] >= atleast]
    count.extend(counter)
    dictionary = dict()
    for word, _ in count:
        dictionary[word] = len(dictionary)
    data = list()
    unk_count = 0
    for word in words:
        index = dictionary.get(word, 0)
        if index == 0:
            unk_count += 1
        data.append(index)
    count[0][1] = unk_count
    reversed_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return data, count, dictionary, reversed_dictionary

def add_start_end(string):
    string = string.split()
    strings = []
    for s in string:
        s = list(s)
        s[0] = '<%s'%(s[0])
        s[-1] = '%s>'%(s[-1])
        strings.extend(s)
    return strings

In [3]:
with open('lemmatization-en.txt','r') as fopen:
    texts = fopen.read().split('\n')
after, before = [], []
for i in texts[:10000]:
    splitted = i.encode('ascii', 'ignore').decode("utf-8").lower().split('\t')
    if len(splitted) < 2:
        continue
    after.append(add_start_end(splitted[0]))
    before.append(add_start_end(splitted[1]))
    
print(len(after),len(before))

10000 10000


In [4]:
concat_from = list(itertools.chain(*before))
vocabulary_size_from = len(list(set(concat_from)))
data_from, count_from, dictionary_from, rev_dictionary_from = build_dataset(concat_from, vocabulary_size_from)
print('vocab from size: %d'%(vocabulary_size_from))
print('Most common words', count_from[4:10])
print('Sample data', data_from[:10], [rev_dictionary_from[i] for i in data_from[:10]])
print('filtered vocab size:',len(dictionary_from))
print("% of vocab used: {}%".format(round(len(dictionary_from)/vocabulary_size_from,4)*100))

vocab from size: 65
Most common words [('e', 9763), ('i', 7229), ('n', 6177), ('s>', 6095), ('o', 5769), ('a', 5633)]
Sample data [46, 5, 11, 14, 35, 47, 4, 6, 10, 39] ['<f', 'i', 'r', 's', 't>', '<t', 'e', 'n', 't', 'h>']
filtered vocab size: 69
% of vocab used: 106.15%


In [5]:
concat_to = list(itertools.chain(*after))
vocabulary_size_to = len(list(set(concat_to)))
data_to, count_to, dictionary_to, rev_dictionary_to = build_dataset(concat_to, vocabulary_size_to)
print('vocab from size: %d'%(vocabulary_size_to))
print('Most common words', count_to[4:10])
print('Sample data', data_to[:10], [rev_dictionary_to[i] for i in data_to[:10]])
print('filtered vocab size:',len(dictionary_to))
print("% of vocab used: {}%".format(round(len(dictionary_to)/vocabulary_size_to,4)*100))

vocab from size: 90
Most common words [('o', 5631), ('a', 5471), ('e', 5471), ('i', 4890), ('r', 4252), ('<c', 4134)]
Sample data [83, 59, 57, 59, 55, 57, 59, 55, 55, 57] ['<1>', '<1', '0>', '<1', '0', '0>', '<1', '0', '0', '0>']
filtered vocab size: 94
% of vocab used: 104.44%


In [6]:
GO = dictionary_from['GO']
PAD = dictionary_from['PAD']
EOS = dictionary_from['EOS']
UNK = dictionary_from['UNK']

In [7]:
for i in range(len(after)):
    after[i].append('EOS')

In [8]:
class Stemmer:
    def __init__(self, size_layer, num_layers, embedded_size, 
                 from_dict_size, to_dict_size, learning_rate, 
                 dropout = 0.5, beam_width = 15):
        
        def lstm_cell(reuse=False):
            return tf.nn.rnn_cell.LSTMCell(size_layer, reuse=reuse)
        
        self.X = tf.placeholder(tf.int32, [None, None])
        self.Y = tf.placeholder(tf.int32, [None, None])
        self.X_seq_len = tf.count_nonzero(self.X, 1, dtype=tf.int32)
        self.Y_seq_len = tf.count_nonzero(self.Y, 1, dtype=tf.int32)
        batch_size = tf.shape(self.X)[0]
        # encoder
        encoder_embeddings = tf.Variable(tf.random_uniform([from_dict_size, embedded_size], -1, 1))
        encoder_embedded = tf.nn.embedding_lookup(encoder_embeddings, self.X)
        encoder_cells = tf.nn.rnn_cell.MultiRNNCell([lstm_cell() for _ in range(num_layers)])
        encoder_dropout = tf.contrib.rnn.DropoutWrapper(encoder_cells, output_keep_prob = 0.5)
        self.encoder_out, self.encoder_state = tf.nn.dynamic_rnn(cell = encoder_dropout, 
                                                                 inputs = encoder_embedded, 
                                                                 sequence_length = self.X_seq_len,
                                                                 dtype = tf.float32)
        
        self.encoder_state = tuple(self.encoder_state[-1] for _ in range(num_layers))
        main = tf.strided_slice(self.Y, [0, 0], [batch_size, -1], [1, 1])
        decoder_input = tf.concat([tf.fill([batch_size, 1], GO), main], 1)
        # decoder
        decoder_embeddings = tf.Variable(tf.random_uniform([to_dict_size, embedded_size], -1, 1))
        decoder_cells = tf.nn.rnn_cell.MultiRNNCell([lstm_cell() for _ in range(num_layers)])
        dense_layer = tf.layers.Dense(to_dict_size)
        training_helper = tf.contrib.seq2seq.ScheduledEmbeddingTrainingHelper(
                inputs = tf.nn.embedding_lookup(decoder_embeddings, decoder_input),
                sequence_length = self.Y_seq_len,
                embedding = decoder_embeddings,
                sampling_probability = 0.5,
                time_major = False)
        training_decoder = tf.contrib.seq2seq.BasicDecoder(
                cell = decoder_cells,
                helper = training_helper,
                initial_state = self.encoder_state,
                output_layer = dense_layer)
        training_decoder_output, _, _ = tf.contrib.seq2seq.dynamic_decode(
                decoder = training_decoder,
                impute_finished = True,
                maximum_iterations = tf.reduce_max(self.Y_seq_len))
        predicting_decoder = tf.contrib.seq2seq.BeamSearchDecoder(
                cell = decoder_cells,
                embedding = decoder_embeddings,
                start_tokens = tf.tile(tf.constant([GO], dtype=tf.int32), [batch_size]),
                end_token = EOS,
                initial_state = tf.contrib.seq2seq.tile_batch(self.encoder_state, beam_width),
                beam_width = beam_width,
                output_layer = dense_layer,
                length_penalty_weight = 0.0)
        predicting_decoder_output, _, _ = tf.contrib.seq2seq.dynamic_decode(
                decoder = predicting_decoder,
                impute_finished = False,
                maximum_iterations = 2 * tf.reduce_max(self.X_seq_len))
        self.training_logits = training_decoder_output.rnn_output
        self.predicting_ids = predicting_decoder_output.predicted_ids[:, :, 0]
        masks = tf.sequence_mask(self.Y_seq_len, tf.reduce_max(self.Y_seq_len), dtype=tf.float32)
        self.cost = tf.contrib.seq2seq.sequence_loss(logits = self.training_logits,
                                                     targets = self.Y,
                                                     weights = masks)
        self.optimizer = tf.train.AdamOptimizer(learning_rate).minimize(self.cost)
        
        y_t = tf.argmax(self.training_logits,axis=2)
        y_t = tf.cast(y_t, tf.int32)
        self.prediction = tf.boolean_mask(y_t, masks)
        mask_label = tf.boolean_mask(self.Y, masks)
        correct_pred = tf.equal(self.prediction, mask_label)
        correct_index = tf.cast(correct_pred, tf.float32)
        self.accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [9]:
size_layer = 128
num_layers = 2
embedded_size = 64
learning_rate = 1e-3
batch_size = 32
epoch = 15

In [10]:
tf.reset_default_graph()
sess = tf.InteractiveSession()
model = Stemmer(size_layer, num_layers, embedded_size, len(dictionary_from), 
                len(dictionary_to), learning_rate,batch_size)
sess.run(tf.global_variables_initializer())

/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [11]:
def str_idx(corpus, dic):
    X = []
    for i in corpus:
        ints = []
        for k in i:
            try:
                ints.append(dic[k])
            except Exception as e:
                ints.append(UNK)
        X.append(ints)
    return X

In [12]:
X = str_idx(before, dictionary_from)
Y = str_idx(after, dictionary_to)

In [13]:
train_X, test_X, train_Y, test_Y = train_test_split(X, Y, test_size = 0.2)

In [14]:
def pad_sentence_batch(sentence_batch, pad_int):
    padded_seqs = []
    seq_lens = []
    max_sentence_len = max([len(sentence) for sentence in sentence_batch])
    for sentence in sentence_batch:
        padded_seqs.append(sentence + [pad_int] * (max_sentence_len - len(sentence)))
        seq_lens.append(len(sentence))
    return padded_seqs, seq_lens

In [15]:
from tqdm import tqdm
from sklearn.utils import shuffle
import time

for EPOCH in range(epoch):
    lasttime = time.time()
    total_loss, total_accuracy, total_loss_test, total_accuracy_test = 0, 0, 0, 0
    train_X, train_Y = shuffle(train_X, train_Y)
    test_X, test_Y = shuffle(test_X, test_Y)
    pbar = tqdm(range(0, len(train_X), batch_size), desc='train minibatch loop')
    for k in pbar:
        index = min(k+batch_size,len(train_X))
        batch_x, seq_x = pad_sentence_batch(train_X[k: k+batch_size], PAD)
        batch_y, seq_y = pad_sentence_batch(train_Y[k: k+batch_size], PAD)
        acc, loss, _ = sess.run([model.accuracy, model.cost, model.optimizer], 
                                      feed_dict={model.X:batch_x,
                                                model.Y:batch_y})
        total_loss += loss
        total_accuracy += acc
        pbar.set_postfix(cost=loss, accuracy = acc)
        
    pbar = tqdm(range(0, len(test_X), batch_size), desc='test minibatch loop')
    for k in pbar:
        index = min(k+batch_size,len(test_X))
        batch_x, seq_x = pad_sentence_batch(test_X[k: k+batch_size], PAD)
        batch_y, seq_y = pad_sentence_batch(test_Y[k: k+batch_size], PAD)
        acc, loss = sess.run([model.accuracy, model.cost], 
                                      feed_dict={model.X:batch_x,
                                                model.Y:batch_y})
        total_loss_test += loss
        total_accuracy_test += acc
        pbar.set_postfix(cost=loss, accuracy = acc)
        
    total_loss /= (len(train_X) / batch_size)
    total_accuracy /= (len(train_X) / batch_size)
    total_loss_test /= (len(test_X) / batch_size)
    total_accuracy_test /= (len(test_X) / batch_size)
        
    print('epoch: %d, avg loss: %f, avg accuracy: %f'%(EPOCH, total_loss, total_accuracy))
    print('epoch: %d, avg loss test: %f, avg accuracy test: %f'%(EPOCH, total_loss_test, total_accuracy_test))

train minibatch loop:   1%|          | 3/250 [00:00<00:09, 25.11it/s, accuracy=0.639, cost=1.27]

epoch: 0, avg loss: 1.756106, avg accuracy: 0.525552
epoch: 0, avg loss test: 1.498321, avg accuracy test: 0.581786


train minibatch loop:   1%|          | 3/250 [00:00<00:08, 28.48it/s, accuracy=0.753, cost=0.88]

epoch: 1, avg loss: 1.253468, avg accuracy: 0.640075
epoch: 1, avg loss test: 1.151150, avg accuracy test: 0.681588


train minibatch loop:   1%|          | 3/250 [00:00<00:08, 27.59it/s, accuracy=0.724, cost=0.966]

...

epoch: 12, avg loss: 0.147347, avg accuracy: 0.959543
epoch: 12, avg loss test: 0.191767, avg accuracy test: 0.955275


train minibatch loop:   1%|          | 3/250 [00:00<00:08, 28.02it/s, accuracy=0.969, cost=0.137] 

epoch: 13, avg loss: 0.124258, avg accuracy: 0.966085
epoch: 13, avg loss test: 0.168982, avg accuracy test: 0.961098


test minibatch loop: 100%|██████████| 63/63 [00:00<00:00, 67.08it/s, accuracy=0.955, cost=0.135] 

epoch: 14, avg loss: 0.110231, avg accuracy: 0.970329
epoch: 14, avg loss test: 0.152320, avg accuracy test: 0.966352


In [16]:
predicted = sess.run(model.predicting_ids, 
                     feed_dict={model.X:batch_x})

In [17]:
for i in range(len(batch_x)):
    print('row %d'%(i+1))
    print('BEFORE:',''.join([rev_dictionary_from[n] for n in batch_x[i] if n not in [0,1,2,3]]))
    print('REAL AFTER:',''.join([rev_dictionary_to[n] for n in batch_y[i] if n not in[0,1,2,3]]))
    print('PREDICTED AFTER:',''.join([rev_dictionary_to[n] for n in predicted[i] if n not in[0,1,2,3]]),'\n')

row 1
BEFORE: <abetting>
REAL AFTER: <abet>
PREDICTED AFTER: <abett> 

row 2
BEFORE: <bruising>
REAL AFTER: <bruise>
PREDICTED AFTER: <bruise> 

row 3
BEFORE: <crash-lands>
REAL AFTER: <crash-land>
PREDICTED AFTER: <crash-land> 

row 4
BEFORE: <calenders>
REAL AFTER: <calender>
PREDICTED AFTER: <calender> 

row 5
BEFORE: <clacking>
REAL AFTER: <clack>
PREDICTED AFTER: <clack> 

row 6
BEFORE: <cooked>
REAL AFTER: <cook>
PREDICTED AFTER: <cook> 

row 7
BEFORE: <corrupted>
REAL AFTER: <corrupt>
PREDICTED AFTER: <corrupt> 

row 8
BEFORE: <coronas>
REAL AFTER: <corona>
PREDICTED AFTER: <corona> 

row 9
BEFORE: <bettings>
REAL AFTER: <betting>
PREDICTED AFTER: <betting> 

row 10
BEFORE: <channelling>
REAL AFTER: <channel>
PREDICTED AFTER: <channell> 

row 11
BEFORE: <clasps>
REAL AFTER: <clasp>
PREDICTED AFTER: <clasp> 

row 12
BEFORE: <carousing>
REAL AFTER: <carouse>
PREDICTED AFTER: <carouse> 

row 13
BEFORE: <bluestones>
REAL AFTER: <bluestone>
PREDICTED AFTER: <bluestone> 

row 14
BEFOR